# Conversión de prescripciones en protocolos clínicos

In [1]:
import aclinprot as acp
import pandas as pd

## Procedimiento
- Exportar la prescripción en ARIA en formato `csv`
- Crear una primera version del protoccolo clínico a partir de la prescripción
- Exportar el archivo de estructuras del paciente
- Cambiar los nombres de las estructuras en la prescripción para hacerlos coincidir con los de contorneo

In [9]:
# Nombre del archivo csv con los datos de la prescripción
prescriptionFile = '../../prescripciones/Pulmon33fx.csv'

# Primera versión del protocolo clínico a partir de la prescripción
acp.convertPrescriptionIntoClinicalProtocol(prescription=prescriptionFile, ProtocolID='testPulmon33fx', 
                                          TreatmentSite='Lungs', PlanID='Pulmon')

# Nombre del archivo de estructuras
structureFile = '../../DICOM/RS.1.2.246.352.71.4.964456977994.133406.20231130083039.dcm'


In [8]:
contStrdf = pd.DataFrame(acp.readContouringStructureNames(structureFile), columns=['Contouring'])
presStrdf = pd.DataFrame(acp.readClinProtStructureNames('ClinicalProtocol.xml'), columns=['Prescription'])
pd.concat([contStrdf, presStrdf], axis=1)

,Contouring,Prescription
0,BODY,PTV 4R
1,Corazón,Lungs
2,CTV 1,medula
3,Esófago,esofago
4,GTV 1,Corazón
5,Médula,NaN
6,PTV 4R,NaN
7,Pulmón Dcho,NaN
8,Pulmón Izqdo,NaN
9,Pulmones,NaN


In [4]:
strNameSuggestions = acp.suggestStrNames(clinprot='ClinicalProtocol.xml',
                                         rsdicom=structureFile)
strNameSuggestions

,Structure,Suggestion
0,PTV prostata,PTV prostata
1,PTV vvss,PTV vvss
2,PTV gg,PTV gg
3,Recto,Recto
4,Vejiga,Vejiga
5,Cabeza Femoral D,Cabeza femoral D
6,Cabeza Femoral I,Cabeza femoral I
7,Intestino,Intestino
